In [1]:
import configparser
import Src.data.dmbioProtDataset as dmbiods
import Src.model.egnn_clean as eg
import torch
import numpy as np

In [2]:
config_path = '/teamspace/studios/this_studio/DeepDive/Prot-DNA_experiments/test/testDatasetConfig.ini'
config = configparser.ConfigParser()
config.read(config_path)
config_section = config['DEFAULT']

In [3]:
dsParams = dmbiods.dmbioProtDataSetParams(
    indir = config_section['indir'],
    in_file = config_section['in_file'],
    label_dir = config_section['label_dir'],
    label_fileExt = config_section['label_fileExt'],
    node_feat_dir = config_section['node_feat_dir'],
    node_feat_fileExt = config_section['node_feat_fileExt'],
    edge_dir = config_section['edge_dir'],
    edge_fileExt = config_section['edge_fileExt'],
    node_cord_dir = config_section['node_cord_dir'],
    node_cord_file_ext = config_section['node_cord_file_ext']
)

In [4]:
datasset = dmbiods.dmbioProtDataSet(dsParams)

In [5]:
data = datasset.getgraphbyid(0)
data

Data(x=[63, 5461], edge_index=[2, 766], edge_attr=[766, 1], y=[63, 1], pos=[63, 3])

In [6]:
egnn = eg.EGNN(in_node_nf=data.x.shape[1], hidden_nf=1028, out_node_nf=1, in_edge_nf=1)

In [7]:
# data.x = data.x.squeeze()
# data.pos = data.pos.squeeze()
# data.edge_index[0] = data.edge_index[0].squeeze()
# data.edge_index[1] = data.edge_index[1].squeeze() 
# data.edge_attr = data.edge_attr.squeeze()
# data.edge_attr = data.edge_attr.unsqueeze(dim=1)

In [8]:
criterion = torch.nn.BCELoss()
h, x = egnn(data.x, data.pos, data.edge_index, data.edge_attr)

In [11]:
pred = torch.nn.Sigmoid()(h)
#pred = pred.detach().numpy()
pred.shape

torch.Size([63, 1])

In [12]:
data.y.shape

torch.Size([63, 1])

In [13]:
l1_loss = criterion(pred, data.y)
l1_loss

tensor(0.6467, grad_fn=<BinaryCrossEntropyBackward0>)

In [14]:
pred = pred.squeeze()
pred = torch.tensor([0 if each<0.5 else 1 for each in pred], dtype=torch.int32)
pred

tensor([1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], dtype=torch.int32)

In [16]:
y_int = data.y.to(torch.int64)
y_int = y_int.squeeze()
t_p = ((pred == 1) & (y_int == 1)).sum()
t_n = ((pred == 0) & (y_int == 0)).sum()
actual_p = (y_int == 1).sum()
t_p, t_n, actual_p

(tensor(10), tensor(36), tensor(14))